In [1]:
#Finding Similar Movies

import pandas as pd

r_cols=['user_id','movie_id','rating']
ratings=pd.read_csv("C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_data.data", sep='\t',names=r_cols,usecols=range(3),encoding='latin-1')
print(ratings.head())
m_cols=['movie_id','title']
movies=pd.read_csv("C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_item.item", sep='|',names=m_cols,usecols=range(2),encoding='latin-1')
print(movies.head())
ratings=pd.merge(movies,ratings)

   user_id  movie_id  rating
0        0        50       5
1        0       172       5
2        0       133       1
3      196       242       3
4      186       302       3
   movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)


In [2]:
print(ratings.head())

   movie_id             title  user_id  rating
0         1  Toy Story (1995)      308       4
1         1  Toy Story (1995)      287       5
2         1  Toy Story (1995)      148       4
3         1  Toy Story (1995)      280       4
4         1  Toy Story (1995)       66       3


In [3]:
#making a user/movie Matrix
movieRatings=ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
print(movieRatings.head())

title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
0                              NaN           NaN                    NaN   
1                              NaN           NaN                    2.0   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
0                        NaN         NaN                          NaN   
1                        5.0         NaN                          NaN   
2                        NaN         NaN                          NaN   
3                        NaN         2.0                          NaN   
4                        NaN        

In [4]:
#extract a series of users who watched StarWars
starWarsRatings=movieRatings['Star Wars (1977)']

print(starWarsRatings.dropna())

user_id
0      5.0
1      5.0
2      5.0
4      5.0
5      4.0
      ... 
937    5.0
938    5.0
940    4.0
942    5.0
943    4.0
Name: Star Wars (1977), Length: 584, dtype: float64


In [5]:
#correlate other movies to star wars
similarMovies=movieRatings.corrwith(starWarsRatings)
similarMovies=similarMovies.dropna()
df=pd.DataFrame(similarMovies)
print(df.head())

c:\Users\Ghost\anaconda3\envs\class\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Ghost\anaconda3\envs\class\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


                                  0
title                              
'Til There Was You (1997)  0.872872
1-900 (1994)              -0.645497
101 Dalmatians (1996)      0.211132
12 Angry Men (1957)        0.184289
187 (1997)                 0.027398


In [6]:
#sort
print(similarMovies.sort_values(ascending=False))

title
Commandments (1997)           1.0
Cosi (1996)                   1.0
No Escape (1994)              1.0
Stripes (1981)                1.0
Man of the Year (1995)        1.0
                             ... 
For Ever Mozart (1996)       -1.0
Frankie Starlight (1995)     -1.0
I Like It Like That (1994)   -1.0
American Dream (1990)        -1.0
Theodore Rex (1995)          -1.0
Length: 1410, dtype: float64


In [7]:
#getting rid of movies that were watched by few people that are producing spurious results
import numpy as np
movieStats=ratings.groupby('title').agg({'rating':[np.size,np.mean]})
print(movieStats.head())

                          rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390


In [8]:
#in up we found movies with >100 is more heard
popularMovies=movieStats['rating']['size']>=300 #initially it was 100
movieStats[popularMovies].sort_values([('rating','mean')],ascending=False)[:15]

rating          
                                            size      mean
title                                                     
Star Wars (1977)                             584  4.359589
Silence of the Lambs, The (1991)             390  4.289744
Godfather, The (1972)                        413  4.283293
Raiders of the Lost Ark (1981)               420  4.252381
Titanic (1997)                               350  4.245714
Empire Strikes Back, The (1980)              368  4.206522
Princess Bride, The (1987)                   324  4.172840
Fargo (1996)                                 508  4.155512
Monty Python and the Holy Grail (1974)       316  4.066456
Pulp Fiction (1994)                          394  4.060914
Fugitive, The (1993)                         336  4.044643
Return of the Jedi (1983)                    507  4.007890
Terminator, The (1984)                       301  3.933555
Indiana Jones and the Last Crusade (1989)    331  3.930514
Full Monty, The (1997)                       315  3.926984

In [9]:
#join similar movies with movies that have >100 ratings
df=movieStats[popularMovies].join(pd.DataFrame(similarMovies,columns=['similarity']))

C:\Users\Ghost\AppData\Local\Temp\ipykernel_11888\2509303253.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df=movieStats[popularMovies].join(pd.DataFrame(similarMovies,columns=['similarity']))


In [10]:
print(df.head())

                                   (rating, size)  (rating, mean)  similarity
title                                                                        
Air Force One (1997)                          431        3.631090    0.113164
Back to the Future (1985)                     350        3.834286    0.274839
Chasing Amy (1997)                            379        3.839050    0.074536
Contact (1997)                                509        3.803536    0.134962
E.T. the Extra-Terrestrial (1982)             300        3.833333    0.303619


In [11]:
df.sort_values(['similarity'],ascending=False)[:20]

,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
E.T. the Extra-Terrestrial (1982),300,3.833333,0.303619
Back to the Future (1985),350,3.834286,0.274839
"Terminator, The (1984)",301,3.933555,0.262255
"Princess Bride, The (1987)",324,4.172840,0.259711
